In [1]:
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
import re

import fasttext 
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
import math
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

Let's start by exploring the succes of the movies 

In [2]:
DATA_FOLDER = '/Users/elieroth/Desktop/MA3/ADA/Project/MovieSummaries/'

CHARACTER_DATASET = DATA_FOLDER+"character.metadata.tsv"
MOVIE_DATASET = DATA_FOLDER+"movie.metadata.tsv"
NAME_DATASET = DATA_FOLDER+"name.clusters.txt"
SUMMARIES_DATASET = DATA_FOLDER+"plot_summaries.txt"
TROPES_DATASET = DATA_FOLDER+"tvtropes.clusters.txt"


df_charact=pd.read_csv(CHARACTER_DATASET, sep='\t', names=['Wikipedia movie ID', 'Freebase movie ID', 'Movie release date', 'Character Name', 'Actor DOB', 'Actor gender', 'Actor height', 'Actor ethnicity', 'Actor Name', 'Actor age at movie release', 'Freebase character map1', 'Freebase character map2', 'Freebase character map3'])
df_movie = pd.read_csv(MOVIE_DATASET, sep='\t', names=['Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genre'])
df_name = pd.read_csv(NAME_DATASET, sep='\t', names=['Character Name', 'Freebase character map'])
df_tropes=pd.read_csv(TROPES_DATASET,sep='\t', header=None, names=['Tropes', 'Characters'])
df_summaries=pd.read_csv(SUMMARIES_DATASET,sep='\t', names=['Wikipedia movie ID', 'Summary'])
trigger_movies = pd.read_csv('EVERY_trigger_movies.csv')
df_movie2 = pd.read_csv('horror_movies.csv')

## Filtering the data to only use the horror genre


In [3]:
# filtering the df to obtain a horro df
df_movie_horror = df_movie.copy(deep=True)
df_movie_horror['genre_names'] = df_movie_horror['Movie genre'].apply(
    lambda genres: [genre.strip() for genre in re.split(r",\s*", genres) if re.search(r'horror', genre, re.IGNORECASE)]
)
df_movie_horror = df_movie_horror[df_movie_horror['genre_names'].astype(bool)] # removing the non horror movie

# Length of the data frames 
print(f'The original size of the CMU dataframe is {len(df_movie)} and the horror one is {len(df_movie_horror)}')
print(f'The size of the horror dataframe is {100*len(df_movie_horror)/len(df_movie):.2f}% of the CMU dataset.')

The original size of the CMU dataframe is 81741 and the horror one is 5313
The size of the horror dataframe is 6.50% of the CMU dataset.


In [4]:
# merging summary dataset and CMU movie dataset
df_movie_horror = pd.merge(df_movie_horror, df_summaries, on=['Wikipedia movie ID'], how='inner')

# cleaning Nan before merging
df_movie_horror=df_movie_horror.dropna(subset="Movie release date").dropna(subset="Summary")
df_movie2.rename(columns={'title': 'Movie name', 'overview':'Summary', 'release_date':'Movie release date'}, inplace=True)
df_movie2=df_movie2.dropna(subset="Movie release date").dropna(subset="Summary")


# concatenate two df
df_horror = pd.concat([df_movie_horror, df_movie2], ignore_index=True)
df_horror = df_horror.groupby('Movie name', as_index=False).first() # merging same movie name and keeping the non null value

# transform the release date of the movie into only years
df_horror["Movie release year"] = df_horror["Movie release date"].str[:4]
print(f'The size of our filtered dataframe is {len(df_horror)}')

The size of our filtered dataframe is 29408


In [5]:
# cleaning the entire dataset for further analysis 

df_movie = pd.merge(df_movie, df_summaries, on=['Wikipedia movie ID'], how='inner')
df_movie=df_movie.dropna(subset="Movie release date").dropna(subset="Summary")
df_movie["Movie release year"] = df_movie["Movie release date"].str[:4]
df_movie['Movie release year'] = pd.to_datetime(df_movie['Movie release year'], errors='coerce', format='%Y')